In [51]:
import boto3
import csv
from cryptography.fernet import Fernet
import json
import base64

In [52]:
iam = boto3.client('iam')
response = response = iam.get_user()
user_name = response['User']['UserName']
user_name

'rspamzn'

In [ ]:
iam = boto3.client("iam")

policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowCreateKey",
            "Effect": "Allow",
            "Action": "kms:*",
            "Resource": "*"
        }
    ]
}

response = iam.create_policy(
    PolicyName = 'AllowCreateKey',
    PolicyDocument = json.dumps(policy_document)
)

policy_arn = response['Policy']['Arn']

In [ ]:
iam.attach_user_policy(
    UserName = user_name,
    PolicyArn = policy_arn
)

In [ ]:
kms = boto3.client('kms')

def create_key():
    response = kms.create_key(
        Description='Key for PII encryption',
        KeyUsage='ENCRYPT_DECRYPT',
        CustomerMasterKeySpec='SYMMETRIC_DEFAULT',
        Origin='AWS_KMS'
    )
    return response['KeyMetadata']['KeyId']

In [ ]:
key_id = create_key()

In [ ]:
key_id

In [53]:
def retrieve_key(key_id):
    response = kms.describe_key(KeyId=key_id)
    return response['KeyMetadata']['Arn']

key = retrieve_key(key_id)

response = kms.generate_data_key(KeyId=key, KeySpec='AES_256')
plaintext_datakey = response['Plaintext']
fernet_key = base64.urlsafe_b64encode(plaintext_datakey)
fernet_key

b'j4fOrbj-7-Cel9t6Kc3642vZbs5uDh9ijUzi1I16CSk='

In [54]:
cipher_suite = Fernet(fernet_key)
cipher_suite

In [55]:
cipher_text = cipher_suite.encrypt("Hello Snickers. Come to me!".encode('utf-8'))
cipher_text


b'gAAAAABnATmiCVHireMgYwx6bgunAfkyuOhz8p2YaJt3an7V_UJhaQhubN0hztYY_lGf2z6ldRFWVykqIezKRo2rXKah_6uf4auJT4j3Y6ETgI-CjRkTj_0='

In [56]:
plain_text = cipher_suite.decrypt(cipher_text).decode('utf-8')
plain_text

'Hello Snickers. Come to me!'